In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('DataFrame').getOrCreate()
#i give it application  name
#i give it master node location and how many cores -->* means all 

21/10/25 20:56:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/25 20:56:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [9]:
sc = spark.sparkContext

In [29]:
lst = [('brooke' , 20) , ( 'denny' , 31) , ('julia' , 30) , ('denny' , 15) , ('julia' , 22) , ('td' , 35) , ('brooke' , 25)]

In [30]:
lst 

[('brooke', 20),
 ('denny', 31),
 ('julia', 30),
 ('denny', 15),
 ('julia', 22),
 ('td', 35),
 ('brooke', 25)]

In [31]:
#we want to do grouping and avg

In [32]:
rdd1 = sc.parallelize(lst)

In [33]:
rdd1.collect()

[('brooke', 20),
 ('denny', 31),
 ('julia', 30),
 ('denny', 15),
 ('julia', 22),
 ('td', 35),
 ('brooke', 25)]

In [34]:
rdd2 = rdd1.map(lambda x : (x[0] ,(x[1] , 1)))

In [35]:
rdd2.collect()

[('brooke', (20, 1)),
 ('denny', (31, 1)),
 ('julia', (30, 1)),
 ('denny', (15, 1)),
 ('julia', (22, 1)),
 ('td', (35, 1)),
 ('brooke', (25, 1))]

In [36]:
rdd3 = rdd2.reduceByKey(lambda x,y : (x[0]+y[0] , x[1]+y[1]))

In [37]:
rdd3.collect()

[('brooke', (45, 2)), ('denny', (46, 2)), ('julia', (52, 2)), ('td', (35, 1))]

In [38]:
rdd4 = rdd3.map(lambda x : (x[0] , x[1][0]/x[1][1]) )

In [39]:
rdd4.collect()

[('brooke', 22.5), ('denny', 23.0), ('julia', 26.0), ('td', 35.0)]

### we will use dataFrames 

In [40]:
from pyspark.sql.functions import avg

In [41]:
lst_df = spark.createDataFrame(lst , ['name' , 'avg'])

In [42]:
lst_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- avg: long (nullable = true)



In [43]:
lst_df

DataFrame[name: string, avg: bigint]

In [44]:
lst_df.show()

+------+---+
|  name|avg|
+------+---+
|brooke| 20|
| denny| 31|
| julia| 30|
| denny| 15|
| julia| 22|
|    td| 35|
|brooke| 25|
+------+---+



In [45]:
avg_df = lst_df.groupBy('name').avg()

In [46]:
avg_df

DataFrame[name: string, avg(avg): double]

In [47]:
avg_df.show()

+------+--------+
|  name|avg(avg)|
+------+--------+
| denny|    23.0|
|brooke|    22.5|
|    td|    35.0|
| julia|    26.0|
+------+--------+



In [48]:
# it is easy and most optimized 

In [49]:
avg_df.collect()#it return list of rows

[Row(name='denny', avg(avg)=23.0),
 Row(name='brooke', avg(avg)=22.5),
 Row(name='td', avg(avg)=35.0),
 Row(name='julia', avg(avg)=26.0)]

In [50]:
avg_df.head()

Row(name='denny', avg(avg)=23.0)

In [51]:
avg_df.describe()

DataFrame[summary: string, name: string, avg(avg): string]

In [52]:
avg_df.describe().show()

+-------+------+-----------------+
|summary|  name|         avg(avg)|
+-------+------+-----------------+
|  count|     4|                4|
|   mean|  null|           26.625|
| stddev|  null|5.793315113128233|
|    min|brooke|             22.5|
|    max|    td|             35.0|
+-------+------+-----------------+



In [53]:
#adv is column structure but in distributed system 

In [54]:
df = spark.read.json('people.json')

In [55]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [56]:
#df op like sql op

In [57]:
df.age

Column<b'age'>

In [58]:
df['age']#the best

Column<b'age'>

In [71]:
from pyspark.sql.functions import col , expr#*
col('age')#very good

Column<b'age'>

In [64]:
df.select('name').show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [65]:
df.select('name' , df['age']*2).show()

+-------+---------+
|   name|(age * 2)|
+-------+---------+
|Michael|     null|
|   Andy|       60|
| Justin|       38|
+-------+---------+



In [66]:
df.select('name' , col('age')*2).show()

+-------+---------+
|   name|(age * 2)|
+-------+---------+
|Michael|     null|
|   Andy|       60|
| Justin|       38|
+-------+---------+



In [67]:
df2 = df.filter(col('age')>21)

In [68]:
df2.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [70]:
df.filter('age > 21').show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [72]:
df.select('name' , expr('age*2')).show()

+-------+---------+
|   name|(age * 2)|
+-------+---------+
|Michael|     null|
|   Andy|       60|
| Justin|       38|
+-------+---------+



In [73]:
df.select('name' , expr('age*2').alias('age_double')).show()

+-------+----------+
|   name|age_double|
+-------+----------+
|Michael|      null|
|   Andy|        60|
| Justin|        38|
+-------+----------+



In [75]:
df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [79]:
df_count = df.groupBy('age').count()

In [80]:
df_count.show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [84]:
lst = [
    ['r' , 'ba' , 1 , 10] , ['b' , 'ba' , 2 , 20] , ['r' , 'ca' , 3 , 30] ,
    ['b' , 'gr' , 4 , 40] , ['r' , 'ca' , 5 , 50] ,['bl' , 'ca' , 6 , 60]
]

In [85]:
df = spark.createDataFrame(lst , schema = ['color' , 'fru' , 'v1' , 'v2'])

In [86]:
df.show()

+-----+---+---+---+
|color|fru| v1| v2|
+-----+---+---+---+
|    r| ba|  1| 10|
|    b| ba|  2| 20|
|    r| ca|  3| 30|
|    b| gr|  4| 40|
|    r| ca|  5| 50|
|   bl| ca|  6| 60|
+-----+---+---+---+



In [89]:
df.printSchema()

root
 |-- color: string (nullable = true)
 |-- fru: string (nullable = true)
 |-- v1: long (nullable = true)
 |-- v2: long (nullable = true)



In [90]:
df.groupBy('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|    b|    3.0|   30.0|
|    r|    3.0|   30.0|
|   bl|    6.0|   60.0|
+-----+-------+-------+



In [91]:
df.groupBy('color').avg('v1').show()

+-----+-------+
|color|avg(v1)|
+-----+-------+
|    b|    3.0|
|    r|    3.0|
|   bl|    6.0|
+-----+-------+



In [92]:
df.groupBy('color' , 'fru').avg('v1' , 'v2').show()

+-----+---+-------+-------+
|color|fru|avg(v1)|avg(v2)|
+-----+---+-------+-------+
|    b| gr|    4.0|   40.0|
|    r| ca|    4.0|   40.0|
|    r| ba|    1.0|   10.0|
|    b| ba|    2.0|   20.0|
|   bl| ca|    6.0|   60.0|
+-----+---+-------+-------+



In [94]:
df.groupBy('color' , 'fru').agg(avg('v1').alias('avf__v1')).show()

+-----+---+-------+
|color|fru|avf__v1|
+-----+---+-------+
|    b| gr|    4.0|
|    r| ca|    4.0|
|    r| ba|    1.0|
|    b| ba|    2.0|
|   bl| ca|    6.0|
+-----+---+-------+



In [95]:
df.groupBy('color' , 'fru').agg(avg('v1').alias('avf__v1'),avg('v2').alias('avf__v2')).show()

+-----+---+-------+-------+
|color|fru|avf__v1|avf__v2|
+-----+---+-------+-------+
|    b| gr|    4.0|   40.0|
|    r| ca|    4.0|   40.0|
|    r| ba|    1.0|   10.0|
|    b| ba|    2.0|   20.0|
|   bl| ca|    6.0|   60.0|
+-----+---+-------+-------+



In [96]:
df.groupBy(['color' , 'fru']).agg(avg('v1').alias('avf__v1'),avg('v2').alias('avf__v2')).show()

+-----+---+-------+-------+
|color|fru|avf__v1|avf__v2|
+-----+---+-------+-------+
|    b| gr|    4.0|   40.0|
|    r| ca|    4.0|   40.0|
|    r| ba|    1.0|   10.0|
|    b| ba|    2.0|   20.0|
|   bl| ca|    6.0|   60.0|
+-----+---+-------+-------+



In [97]:
#if you give it schema it will be better